# 学習プログラム

## path通し

In [1]:
import sys
import os

sys.path.append(os.path.split(os.getcwd())[0])


## データプレプロセス

## vocablary作成
すでにvocab.txtが作成済みである場合不用

In [2]:
from fast_jtnn.mol_tree import MolTree
from getpass import getpass,getuser

import mysql
from mysql import connector
import warnings

# SQL server profile
host = "localhost"
user = None
passwd = None
port = 3306
database="chemoinfo"

# 
VOCAB_FILE = "./MS_vocab.txt"

# get massbank data from SQL server
try:
    if not isinstance(user,str):
        user = raw_input("user")
    if not isinstance(passwd,str):
        passwd = getpass()
    connect = connector.connect(host=host,user=user,password=passwd,port=port,database=database)
    cursor = connect.cursor()
    cursor.execute("""select smiles from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A'; """)
    smiles_list = cursor.fetchall()
except mysql.connector.Error as e:
    print("Something went wrong: {}".format(e))
    sys.exit(1)
finally:
    if passwd : del passwd
    if connect: connect.close()
    if cursor: cursor.close()

# create vocablary
succes = 0
fault = 0
cset = set()
for one in smiles_list:
    try:
        mol = MolTree(one[0])
    except AttributeError as e:
        warnings.warn("Entered An SMILES that does not meet the rules")
        continue
    for c in mol.nodes:
        cset.add(c.smiles)

# write vocab
with open(VOCAB_FILE,"w") as f:
    for one in cset:
        f.write(one+"\n")

user aisiars
 


Something went wrong: 1045 (28000): Access denied for user 'aisiars'@'localhost' (using password: NO)


NameError: name 'connect' is not defined

! mkdir vae_model/
%run ../fast_molvae/vae_train.py --train processed --vocab ./MS_vocab.txt --save_dir vae_model/

# Vocab,datasetのロード

In [2]:
from fast_jtnn import *
from MS_PredictModel import MS_Dataset,MS_Dataset_pickle
import pickle

VOCAB_FILE = "./MS_vocab.txt"

vocab = [x.strip("\r\n ") for x in open(VOCAB_FILE,"r")]
vocab = Vocab(vocab)

'''
MS_Dataset.QUERY = """select smiles,file_path from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A';"""
dataset = MS_Dataset(vocab=vocab,host="localhost",database="chemoinfo",batch_size=20)
'''

dataset=MS_Dataset_pickle("./massbank.pkl",vocab,batch_size=20)

## モデルの作成

In [3]:
from ms_encoder import ms_peak_encoder,ms_peak_encoder_lstm
import torch.nn as nn
import torch
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 3

dec_model = JTNNVAE(vocab, hidden_size, latent_size, depthT, depthG).cuda()
print dec_model
enc_model = ms_peak_encoder_lstm(dataset.max_spectrum_size,latent_size).cuda()
print enc_model

for param in dec_model.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)
load_model = "./vae_model/model.iter-70000"
dec_model.load_state_dict(torch.load(load_model))
print "Model #Params: %dK" % (sum([x.nelement() for x in dec_model.parameters()]) / 1000,)
print "Model #Params: %dK" % (sum([x.nelement() for x in enc_model.parameters()]) / 1000,)

/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


JTNNVAE(
  (jtnn): JTNNEncoder(
    (embedding): Embedding(365, 100)
    (outputNN): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (GRU): GraphGRU(
      (W_z): Linear(in_features=200, out_features=100, bias=True)
      (W_r): Linear(in_features=100, out_features=100, bias=False)
      (U_r): Linear(in_features=100, out_features=100, bias=True)
      (W_h): Linear(in_features=200, out_features=100, bias=True)
    )
  )
  (decoder): JTNNDecoder(
    (embedding): Embedding(365, 100)
    (W_z): Linear(in_features=200, out_features=100, bias=True)
    (U_r): Linear(in_features=100, out_features=100, bias=False)
    (W_r): Linear(in_features=100, out_features=100, bias=True)
    (W_h): Linear(in_features=200, out_features=100, bias=True)
    (W): Linear(in_features=128, out_features=100, bias=True)
    (U): Linear(in_features=128, out_features=100, bias=True)
    (U_i): Linear(in_features=200, out_features=100, bias=True)
    (W_o): 

## オプティマイザの設定

In [4]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(enc_model.parameters(), lr=1e-3)
#optimizer = optim.SGD(enc_model.parameters(),lr=100)
scheduler = lr_scheduler.ExponentialLR(optimizer, 0.9)
scheduler.step()

In [ ]:
from MS_PredictModel import ms_peak_encoder,MS_Dataset
from tqdm import tqdm
import numpy as np

pbar = None
dataset.batch_size = 10
def training(max_epoch = 100):
    global pbar
    total_step = 0
    meters = np.zeros(3)
    for epoch in range(max_epoch):
        print("epoch : ",epoch)
        for batch in dataset:
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            total_step+=1
            #pbar.update(1)
            enc_model.zero_grad()
            dec_model.zero_grad()
            h = enc_model(x,y)
            tree_vec = h[:,:h.shape[1]/2]
            mol_vec  = h[:,h.shape[1]/2:]
            _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
            word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
            assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
            total_loss = word_loss+topo_loss+assm_loss
            total_loss.backward()
            optimizer.step()
            
            meters = meters + np.array([word_acc * 100, topo_acc * 100, assm_acc * 100])
            if total_step % 200 == 0:
                meters /= 200
                print "[%d] , Word: %.2f, Topo: %.2f, Assm: %.2f" % (total_step,meters[0], meters[1], meters[2])
                sys.stdout.flush()
                meters *= 0
            if total_step % 200 == 0:
                torch.save(enc_model.state_dict(), "./enc_model" + "/model.iter-" + str(total_step))

#import pdb; pdb.set_trace()
try:
    #if pbar is None:
        #pbar = tqdm()
    training(100)
except RuntimeError as e:
    #if pbar is not None:
        #del pbar
    import traceback
    print(traceback.format_exc())
    import pdb; pdb.set_trace()
    print(e)



('epoch : ', 0)


/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[200] , Word: 45.54, Topo: 87.71, Assm: 87.28
[400] , Word: 46.42, Topo: 87.58, Assm: 89.05
[600] , Word: 46.55, Topo: 87.54, Assm: 89.02
('epoch : ', 1)
[800] , Word: 48.44, Topo: 88.25, Assm: 89.86
[1000] , Word: 48.46, Topo: 88.24, Assm: 89.12
[1200] , Word: 48.99, Topo: 88.04, Assm: 89.17
[1400] , Word: 48.66, Topo: 88.06, Assm: 88.53
('epoch : ', 2)
[1600] , Word: 50.04, Topo: 88.57, Assm: 89.63
[1800] , Word: 49.39, Topo: 88.85, Assm: 88.31
[2000] , Word: 50.12, Topo: 89.44, Assm: 89.38
[2200] , Word: 51.84, Topo: 90.28, Assm: 89.58
('epoch : ', 3)
[2400] , Word: 49.59, Topo: 89.77, Assm: 88.19
[2600] , Word: 50.56, Topo: 90.66, Assm: 89.67
[2800] , Word: 50.76, Topo: 90.42, Assm: 89.62
('epoch : ', 4)
[3000] , Word: 52.60, Topo: 90.74, Assm: 89.92
[3200] , Word: 52.30, Topo: 91.02, Assm: 90.43
[3400] , Word: 50.59, Topo: 90.58, Assm: 89.33
[3600] , Word: 50.78, Topo: 90.67, Assm: 89.23
('epoch : ', 5)
[3800] , Word: 51.65, Topo: 90.74, Assm: 89.40
[4000] , Word: 51.53, Topo: 90.

In [42]:
for data in dataset:
    x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = data
    data = torch.stack((x,y),2)
    print(data[0])
    indc = data[:,:,1].argsort(1,descending=True)
    print(indc.size())
    data = [torch.index_select(data[batch,:,:],0,indc[batch,:]) for batch in range(data.size()[0])]
    print(data)
    break

tensor([[ 15.0000,   1.0700],
        [ 26.0000,   2.3500],
        [ 27.0000,  27.0700],
        [ 29.0000,  32.9900],
        [ 30.0000,   1.2000],
        [ 31.0000,   2.7400],
        [ 39.0000,  14.0000],
        [ 41.0000,  49.8200],
        [ 42.0000,  11.2000],
        [ 43.0000,  49.3900],
        [ 44.0000,   1.2300],
        [ 45.0000,   2.4800],
        [ 53.0000,   1.2200],
        [ 55.0000,   5.7400],
        [ 56.0000,  11.3800],
        [ 57.0000,  99.9900],
        [ 58.0000,   4.6300],
        [ 59.0000,   4.9900],
        [ 69.0000,   1.2700],
        [ 73.0000,   3.5700],
        [ 74.0000,  25.0000],
        [ 75.0000,   3.5700],
        [ 85.0000,  68.2900],
        [ 86.0000,   4.4400],
        [ 87.0000,  12.8100],
        [102.0000,   3.6500],
        [103.0000,  70.0500],
        [104.0000,   4.1900],
        [115.0000,   1.9600],
        [116.0000,  12.2400],
        [ -1.0000,  -1.0000],
        [ -1.0000,  -1.0000],
        [ -1.0000,  -1.0000],
        [ 